In [1]:
#loss = bias + variance + noise
#use for regression perfomance measure

In [2]:
import pandas as pd

In [16]:
original = pd.read_csv('../beijing.csv', encoding="gbk")

/tmp/ipykernel_117896/3258526607.py:1: DtypeWarning: Columns (1,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  original = pd.read_csv('../beijing.csv', encoding="gbk")


In [17]:
original["livingRoom"].unique()

array([2, 3, 1, 4, 5, 6, 9, 0, 7, 8, '1', '4', '3', '2', '5', '7', '9',
       '6', '0', '#NAME?', '8'], dtype=object)

In [7]:
cleaned = pd.read_csv('../app/data/01_pre_cleaned.csv', sep=';')

In [8]:
cleaned

,Unnamed: 0,floorType,floor,Lng,Lat,totalPrice,square,livingRoom,drawingRoom,kitchen,...,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage,tradeYear,tradeMonth,tradeDay
0,0,1,26,116.475489,40.019520,415.0,131.00,2,1,1,...,6,0.217,1.0,0.0,1.0,7,56021.0,2016,8,9
1,1,1,22,116.453917,39.881534,575.0,132.38,2,2,1,...,6,0.667,1.0,1.0,0.0,7,71539.0,2016,7,28
2,2,2,4,116.561978,39.877145,1030.0,198.00,3,2,1,...,6,0.500,1.0,0.0,0.0,7,48160.0,2016,12,11
3,3,3,21,116.438010,40.076114,297.5,134.00,3,1,1,...,6,0.273,1.0,0.0,0.0,6,51238.0,2016,9,30
4,4,2,6,116.428392,39.886229,392.0,81.00,2,1,1,...,2,0.333,0.0,1.0,1.0,1,62588.0,2016,8,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318811,318846,2,5,116.546899,39.755236,350.0,170.07,3,2,1,...,6,0.333,0.0,1.0,0.0,3,36545.0,2016,6,10
318812,318847,2,24,116.497474,39.810115,108.8,35.09,1,0,1,...,6,0.222,1.0,0.0,1.0,3,54282.0,2016,6,10
318813,318848,2,7,116.497256,39.804081,359.0,102.17,2,2,1,...,6,0.500,1.0,1.0,0.0,3,46927.0,2016,6,5
318814,318849,2,5,116.501794,39.799347,720.0,178.34,4,2,1,...,2,0.500,0.0,1.0,0.0,3,54842.0,2016,6,12


In [3]:
trainDF = pd.read_csv('../app/data/02_train.csv', sep=';')
testDF = pd.read_csv('../app/data/02_test.csv', sep=';')

In [5]:
trainDF

,floorType,floor,Lng,Lat,square,livingRoom,drawingRoom,kitchen,bathRoom,buildingType,...,elevator,fiveYearsProperty,subway,district,communityAverage,tradeYear,tradeMonth,tradeDay,constructionTimePeriod,totalPrice
0,84113,6,116.474283,39.930035,51.82,2,1,1,1,120872,...,0.0,1.0,1.0,7,67222.0,2015,10,9,16727,201.0
1,84113,32,116.416156,40.082585,97.78,2,1,1,1,59076,...,1.0,0.0,1.0,6,33820.0,2011,11,13,139258,142.0
2,44810,6,116.520858,39.918846,61.13,2,1,1,1,120872,...,0.0,1.0,1.0,7,56752.0,2015,4,12,13522,200.0
3,44810,6,116.350909,39.745849,62.82,2,1,1,1,120872,...,0.0,1.0,0.0,4,38492.0,2014,8,24,13522,138.0
4,49078,6,116.351933,39.887523,68.89,2,1,1,1,120872,...,0.0,1.0,0.0,10,87885.0,2016,2,2,139258,440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223166,84113,13,116.416187,39.969551,119.16,2,2,1,1,41709,...,1.0,1.0,0.0,1,92338.0,2013,1,4,139258,430.0
223167,84113,7,116.298972,39.943157,119.07,2,2,1,2,120872,...,1.0,1.0,1.0,8,87420.0,2016,5,9,139258,886.0
223168,26053,13,116.440460,39.860650,120.14,3,1,1,2,120872,...,1.0,1.0,0.0,2,57690.0,2016,4,30,52909,473.0
223169,49078,19,116.368316,39.892214,74.61,2,1,1,1,59076,...,1.0,1.0,1.0,10,75806.0,2014,4,28,139258,340.0
